# SARSA en Machine Learning - Parte I

El acrónimo SARSA proviene de las siglas en inglés de la expresión: _estado, acción recompensa, estado, acción_. Es decir: _State, Action, Reward, State, Action_.

SARSA es un algoritmo que se utiliza para **aprender la política de un agente**, entendiendo como **mejor política** la _mejor acción que se pueda tomar en cada estado o situación_.

SARSA es bastante similar a Q-Learning, pero con la diferencia de que mientras Q-Learning solamente considera el estado actual del agente (en qué casilla estaba el robot, por recordar el ejemplo visto en Q-Learning), **SARSA toma en cuenta tanto el estado actual como la acción actual para predecir la próxima acción y su recompensa**.

Este enfoque se conoce como **aprendizaje on policy**, donde el agente **aprende el valor de la política** que está siendo seguida actualmente, incluyendo cómo va a decidir sus acciones futuras.

La actualización de los valores Q en SARSA sigue prácticamente la misma fórmula vista en Q-Learning (ecuación de Bellman), con la diferencia de que la recompensa observada se indica por una `r` (minúscula).

![SARSA.png](./assets/SARSA.png)

## Caso de ejemplo

In [1]:
import itertools as it
import logging as lg
import random as rnd

import numpy as np
import matplotlib.pyplot as plt

In [2]:
grid: tuple[int, int] = (4, 4)

initial_state: tuple[int, int] = (0, 0)
goal_state: tuple[int, int] = (3, 3)

actions: list[tuple[int, int]] = [
    (0, -1),    # up
    (0, 1),     # down
    (-1, 0),    # left
    (1, 0),     # right
]

action_symbols: list[str] = ['↑', '↓', '→', '←']

In [3]:
max_possible_states: int = grid[0] * grid[1]
max_possible_states

16

In [4]:
max_possible_actions: int = len(actions)
max_possible_actions

4

In [5]:
Q = np.zeros((max_possible_states, max_possible_actions))
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [6]:
# SARSA parameters
alpha: float = 0.1
gamma: float = 0.99
epsilon: float = 0.2
episodes: int = 1000

In [7]:
def convert_state_to_index(
        state: tuple[int, int],
        grid: tuple[int, int]
        ) -> int:
    """
    Convert the current two-dimensional representation of the current
    state (the agent's position on the grid) to an unique linear index.

    Every possible state can be represented as a index number for the
    Q table.

    Parameters
    ----------
    state : tuple[int, int]
        The current two-dimensional representation of the current state
        (the agent's position on the grid).

    grid : tuple[int, int]
        The width and height of the grid.

    Returns
    -------
    int
        The index of the current state in the Q table.
    
    Examples
    --------
    >>> convert_state_to_index((0, 0), (5, 5))
    0
    >>> convert_state_to_index((4, 4), (5, 5))
    24

    """
    return state[0] * grid[1] + state[1]

In [8]:
example: int = convert_state_to_index((3, 0), grid)
example

12

Para entender mejor el resultado de `example`:

![./assets/ejemplo_cuadricula_4x4.png](./assets/4x4_grid_example.png)

In [10]:
def choose_action(
        Q: np.ndarray,
        state: tuple[int, int],
        max_actions: int,
        grid: tuple[int, int],
        epsilon: float
        ) -> int:
    """
    Choose a random action or the best action for the current state,
    depending on the exploration rate.

    Parameters
    ----------
    Q : np.ndarray
        The Q table.
    
    state : tuple[int, int]
        The current two-dimensional representation of the current state
        (the agent's position on the grid).
    
    max_actions : int
        The maximum possible actions for the agent. 

    grid : tuple[int, int]
        The width and height of the grid.

    epsilon : float
        The exploration rate.

    Returns
    -------
    int
        A random action or the best action for the current state,
        depending on the exploration rate.
    
    Requirements
    ------------
    1. Python's built-in `random` library (`import random as rnd`).
    2. NumPy (`import numpy as np`).
    3. The `convert_state_to_index()` function declared above.
    """
    random_number: float = rnd.uniform(0, 1)
    random_action: int = rnd.randint(0, max_actions - 1)
    best_action: int = int(np.argmax(Q[convert_state_to_index(state, grid)]))

    if random_number < epsilon:
        return random_action
    return best_action

In [11]:
def apply_action(
        action: int,
        state: tuple[int, int],
        goal_state: tuple[int, int],
        grid: tuple[int, int],
        obstacles: list[tuple[int, int]]
        ) -> tuple[tuple[int, int], int, bool]:
    """
    Apply the action to the current state.

    Parameters
    ----------
    action : int
        The action to be applied.

    state : tuple[int, int]
        The current two-dimensional representation of the current state
        (the agent's position on the grid).

    goal_state : tuple[int, int]
        The two-dimensional representation of the state declared as
        goal.

    grid : tuple[int, int]
        The width and height of the grid.

    obstacles : list[tuple[int, int]]
        The list of obstacles on the grid.

    Returns
    -------
    tuple[tuple[int, int], int, bool]
        The new state, the reward, and whether the game is over.

    Requirements
    ------------
    - NumPy (`import numpy as np`).
    """
    reward: int = 0
    new_state: tuple[int, int] = tuple(np.add(state, action) % np.array(grid))
    is_game_over: bool = False

    if new_state == goal_state:
        reward = 1
        is_game_over: bool = True
    else:
        reward = -1
    
    return new_state, reward, is_game_over
    